In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

from rwanda.src.utils import get_repo_root
from rwanda.src.impute import impute_data
from rwanda.src.reduce import reduce_dimenions


In [3]:
# Read training data
train = pd.read_csv(f'{get_repo_root()}/data/train.csv')
# Read test data
test = pd.read_csv(f'{get_repo_root()}/data/test.csv')

Make ID and datevar

In [4]:
# From data frame train, extract from ID_LAT_LON_YEAR_WEEK column ID_LAT_LON as a new column
train['ID_LAT_LON'] = ["_".join(x.split('_')[0:3]) for x in train['ID_LAT_LON_YEAR_WEEK']]
test['ID_LAT_LON'] = ["_".join(x.split('_')[0:3]) for x in test['ID_LAT_LON_YEAR_WEEK']]
# From data frame train, extract from ID_LAT_LON_YEAR_WEEK column YEAR_WEEK as a new column
train["year_week"] = ["_".join(x.split('_')[3:5]) for x in train['ID_LAT_LON_YEAR_WEEK']]
test["year_week"] = ["_".join(x.split('_')[3:5]) for x in test['ID_LAT_LON_YEAR_WEEK']]


Imputation

Dim Reduction

Preprocessing

In [7]:
#Dummify ID_LAT_LON
train = pd.get_dummies(train, columns=['ID_LAT_LON'])
test = pd.get_dummies(test, columns=['ID_LAT_LON'])

Timeseries

In [8]:
from rwanda.src.ts_estimate import get_time_series_prediction
from tqdm import tqdm

# Initialize empty data frame with ID_LAT_LON_YEAR_WEEK and emission columns
submission = pd.DataFrame(columns=["ID_LAT_LON_YEAR_WEEK", "emission"])

for id in tqdm(set(train["ID_LAT_LON"])):
    tmp_train = train[train["ID_LAT_LON"] == id]
    tmp_test = test[test["ID_LAT_LON"] == id]
    res = get_time_series_prediction(tmp_train, tmp_test)
    # Concat res to submission
    submission = pd.concat([submission, res], axis=0)

# Make submission
# Make emission to 0 when it is negative
submission.loc[submission["emission"] < 0, "emission"] = 0
submission.to_csv("submission.csv", index=False)